# **ECS7020P Mini Project Advanced**
**Student Name: Aakaash Balasubramanian**

**Student ID: 230199668**

# **Problem Formulation**
The machine learning problem addressed here is a multi-class classification task that focuses specifically on predicting the top 5 cuisines. The objective is to determine the cuisine type of a dish based on its ingredient list. The dataset contains images labeled with cuisine types and corresponding ingredient lists. Successfully solving this problem involves developing a model that excels at predicting the cuisine categories of dishes, with a particular emphasis on the top 5 cuisines. This refined approach aims to enhance the accuracy and interpretability of the model by concentrating on the most prevalent and discernible culinary traditions. The model's success would provide valuable insights into the cultural and culinary associations of the selected top cuisines.

# **Machine Learning pipeline**
**Input:** Raw dataset in a compressed format.

**1.Data Download and Extraction:** Dataset is extracted to a readable format.

**2.Data Preprocessing:** Processed dataset with handled missing values and transformed text data (ingredients) using CountVectorizer.

**3.Modeling - Initial Attempt:** Built a Random Forest classifier to predict cuisine types. Split the dataset into training and testing sets. Evaluated the model's performance, which initially yielded low accuracy due to the multitude of cuisines.

**4.Class Weighted Reduction:** Dropped cuisines with a limited number of data points to focus on the top 5 cuisines. Adjusted class weights to give less importance to Indian cuisine during model training.

**5.Modeling - Final Attempt** Rebuilt the Random Forest classifier with the modified dataset. Split the dataset into training and testing sets. Evaluated the final model's performance, which demonstrated increased accuracy.

**Output:** The type of cuisine based on ingredients.

Download and extract the data. Install necessary libraries

In [1]:
pip install mlend --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00


In [3]:
import mlend
from mlend import download_yummy, yummy_load

subset = {}

datadir = download_yummy(save_to = '/content/drive/MyDrive/Data/MLEnd', subset = subset,verbose=1,overwrite=False)

100%|▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓|3250\3250|003250.jpg
Done!


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# **Dataset**

The MLEND Yummy Dataset is a comprehensive collection of culinary information, featuring various attributes related to dishes, cuisine types, and visual representations through images. The dataset is structured as a DataFrame with 3250 rows and 12 columns, each corresponding to different aspects of the culinary data.

In [5]:
# Load the dataset
dataset_path = '/content/drive/MyDrive/Data/MLEnd/yummy/MLEndYD_image_attributes_benchmark.csv'
df = pd.read_csv(dataset_path)

In [6]:
df

,filename,Diet,Cuisine_org,Cuisine,Dish_name,Home_or_restaurant,Ingredients,Healthiness_rating,Healthiness_rating_int,Likeness,Likeness_int,Benchmark_A
0,000001.jpg,non_vegetarian,japanese,japanese,chicken_katsu_rice,marugame_udon,"rice,chicken_breast,spicy_curry_sauce",neutral,3.0,like,4.0,Train
1,000002.jpg,non_vegetarian,english,english,english_breakfast,home,"eggs,bacon,hash_brown,tomato,bread,tomato,bake...",unhealthy,2.0,like,4.0,Train
2,000003.jpg,non_vegetarian,chinese,chinese,spicy_chicken,jinli_flagship_branch,"chili,chicken,peanuts,sihuan_peppercorns,green...",neutral,3.0,strongly_like,5.0,Train
3,000004.jpg,vegetarian,indian,indian,gulab_jamun,home,"sugar,water,khoya,milk,salt,oil,cardamon,ghee",unhealthy,2.0,strongly_like,5.0,Train
4,000005.jpg,non_vegetarian,indian,indian,chicken_masala,home,"chicken,lemon,turmeric,garam_masala,coriander_...",healthy,4.0,strongly_like,5.0,Train
...,...,...,...,...,...,...,...,...,...,...,...,...
3245,003246.jpg,vegetarian,indian,indian,zeera_rice,home,"1_cup_basmati_rice,2_cups_water,2_tablespoons_...",healthy,4.0,strongly_like,5.0,Train
3246,003247.jpg,vegetarian,indian,indian,paneer_and_dal,home,"fried_cottage_cheese,ghee,lentils,milk,wheat_f...",healthy,4.0,strongly_like,5.0,Test
3247,003248.jpg,vegetarian,indian,indian,samosa,home,"potato,onion,peanut,salt,turmeric_powder,red_c...",very_unhealthy,1.0,like,4.0,Test
3248,003249.jpg,vegan,indian,indian,fruit_milk,home,"kiwi,banana,apple,milk",very_healthy,5.0,strongly_like,5.0,Train


Prepare the data for a machine learning model by handling missing values, checking for NaN values, and transforming the text data (ingredients) into a numerical format using the CountVectorizer.

# **Transformation stage**
Several transformations were applied to the dataset to refine it for model training. First, any missing values in the dataset were removed to ensure data completeness. Additionally, the textual data representing ingredients was transformed into a numerical format using vectorization, specifically the CountVectorizer. This conversion enables the machine learning model to interpret and learn from the ingredient data, facilitating the classification of cuisines based on their characteristic ingredients. The chosen transformations aim to optimize the dataset for effective model training, emphasizing relevant cuisines and representing textual data in a numerical format that aligns with the requirements of the machine learning algorithm.

In [7]:
df['Ingredients'] = df['Ingredients'].fillna('')  # Fill NaN values with an empty string
df['Cuisine_org'] = df['Cuisine_org'].fillna('')  # Fill NaN values with an empty string

# Check for NaN values in the DataFrame
print("NaN values in 'Ingredients':", df['Ingredients'].isnull().sum())
print("NaN values in 'Cuisine_org':", df['Cuisine_org'].isnull().sum())

# Feature Engineering
X = df['Ingredients']
y = df['Cuisine_org']

vectorizer = CountVectorizer()
X_transformed = vectorizer.fit_transform(X)

NaN values in 'Ingredients': 0
NaN values in 'Cuisine_org': 0


# **Modelling**
The chosen machine learning model for this task is the Random Forest Classifier. This decision is grounded in the algorithm's robust performance in handling multi-class classification problems, particularly when dealing with a diverse set of cuisines derived from ingredient data. Random Forests excel in capturing intricate patterns within datasets, offering a high degree of accuracy and resilience to overfitting. Additionally, their ensemble nature, combining multiple decision trees, makes them adept at handling complex relationships between ingredients and cuisine types. The inherent ability to assess feature importance aids in interpreting the significance of different ingredients in predicting cuisine, contributing to a more interpretable and insightful model. Overall, the Random Forest Classifier was selected for its versatility, effectiveness in multi-class classification, and suitability for capturing intricate patterns in the ingredient-based dataset.

# **Methodology**
The training and validation of the models involve a systematic process to ensure robust performance and accurate predictions. The dataset is split into training and testing sets using a train-test split.The Random Forest Classifier is then trained on the training set, learning the relationships between ingredients and cuisine types. Model performance is assessed using a variety of metrics, including accuracy, which measures the overall correctness of predictions. The confusion matrix provides a detailed breakdown of true positive, true negative, false positive, and false negative predictions, offering insights into specific classification errors. Precision, recall, and F1 score complement accuracy by quantifying the model's precision in positive predictions, its ability to capture all relevant instances, and the balance between precision and recall, respectively. These metrics collectively offer a comprehensive evaluation of the model's predictive capabilities, enabling a nuanced understanding of its strengths and potential areas for improvement.

Building, training, evaluating, and saving a machine learning model.

In [8]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Model Selection and Training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))

# Save the trained model for future use
model_filename = '/content/drive/MyDrive/Data/MLEnd/recipe_classifier_model.pkl'
joblib.dump(model, model_filename)
print(f'Model saved to {model_filename}')

# Inference (Predicting cuisine type for new dishes)
# Assuming 'new_dishes' is a list of dishes with their ingredients
new_dishes = [
    "pasta, tomato sauce, cheese",
    "sushi rice, nori, salmon, avocado",
    "chicken, curry powder, coconut milk, potatoes"
]

# Transform the new dishes using the same vectorizer
new_dishes_transformed = vectorizer.transform(new_dishes)

# Make predictions
predictions = model.predict(new_dishes_transformed)

# Display the predictions
for dish, prediction in zip(new_dishes, predictions):
    print(f'Dish: {dish}, Predicted Cuisine: {prediction}')

Accuracy: 0.53
                       precision    recall  f1-score   support

                            1.00      1.00      1.00         1
              afghani       0.00      0.00      0.00         2
              african       0.00      0.00      0.00         1
              america       0.50      0.50      0.50         2
             american       0.49      0.52      0.50        58
     american_cuisine       0.00      0.00      0.00         2
                 arab       1.00      0.50      0.67         2
                asian       0.00      0.00      0.00         3
           australian       0.00      0.00      0.00         1
                azeri       0.50      1.00      0.67         1
          bangladeshi       0.00      0.00      0.00         2
              belgian       0.00      0.00      0.00         1
              british       0.24      0.21      0.22        38
            bulgarian       0.00      0.00      0.00         2
                china       0.00      0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control thi

Model saved to /content/drive/MyDrive/Data/MLEnd/recipe_classifier_model.pkl
Dish: pasta, tomato sauce, cheese, Predicted Cuisine: italian
Dish: sushi rice, nori, salmon, avocado, Predicted Cuisine: japanese
Dish: chicken, curry powder, coconut milk, potatoes, Predicted Cuisine: indian


The low accuracy observed in the model evaluation results is primarily due to the presence of a large number of diverse cuisines in the dataset. The model is struggling to accurately predict less prevalent cuisines, resulting in a low overall accuracy.

To focus the analysis, the dataset has been refined to include only the top 5 cuisines. The updated distribution shows the count of recipes for each of the cuisines. This reduction in the number of cuisines is intended to simplify the classification task, potentially improving the model's ability to predict cuisine types accurately, especially for the more prevalent categories. The refined dataset with the top 5 cuisines will be used for further analysis and model training.

In [11]:
# Print the distribution of cuisines
cuisine_distribution = df['Cuisine_org'].value_counts()
print("Cuisine Distribution:")
print(cuisine_distribution)

# Select the top 4 or 5 cuisines
top_cuisines = cuisine_distribution.head(5).index.tolist()

# Filter the dataset for the selected cuisines
df_filtered = df[df['Cuisine_org'].isin(top_cuisines)]

# Print the updated distribution
print("\nTop Cuisines:")
print(df_filtered['Cuisine_org'].value_counts())

Cuisine Distribution:
indian            1102
chinese            332
italian            279
american           239
british            193
                  ... 
argentina            1
middle-east          1
united_states        1
tropical             1
german/turkish       1
Name: Cuisine_org, Length: 182, dtype: int64

Top Cuisines:
indian      1102
chinese      332
italian      279
american     239
british      193
Name: Cuisine_org, dtype: int64


In [10]:
# Filter the dataset for the top 5 cuisines
top_cuisines = ['indian', 'chinese', 'italian', 'american', 'british']
df_filtered = df[df['Cuisine_org'].isin(top_cuisines)]

# Print the updated distribution
print("\nTop Cuisines:")
print(df_filtered['Cuisine_org'].value_counts())

# Feature Engineering on the filtered dataset
X_filtered = df_filtered['Ingredients']
y_filtered = df_filtered['Cuisine_org']

vectorizer = CountVectorizer()
X_transformed_filtered = vectorizer.fit_transform(X_filtered)

# Split the filtered dataset
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(
    X_transformed_filtered, y_filtered, test_size=0.2, random_state=42
)

# Model Selection and Training on the filtered dataset
model_filtered = RandomForestClassifier()
model_filtered.fit(X_train_filtered, y_train_filtered)

# Model Evaluation on the filtered dataset
y_pred_filtered = model_filtered.predict(X_test_filtered)
accuracy_filtered = accuracy_score(y_test_filtered, y_pred_filtered)

print(f'Accuracy on Filtered Dataset: {accuracy_filtered:.2f}')

# Display classification report on the filtered dataset
print(classification_report(y_test_filtered, y_pred_filtered))


Top Cuisines:
indian      1102
chinese      332
italian      279
american     239
british      193
Name: Cuisine_org, dtype: int64
Accuracy on Filtered Dataset: 0.74
              precision    recall  f1-score   support

    american       0.81      0.51      0.62        57
     british       0.44      0.33      0.38        36
     chinese       0.55      0.71      0.62        70
      indian       0.83      0.89      0.86       211
     italian       0.82      0.73      0.77        55

    accuracy                           0.74       429
   macro avg       0.69      0.63      0.65       429
weighted avg       0.75      0.74      0.74       429



Filtering the dataset to focus on the top 5 cuisines—American, British, Chinese, Indian, and Italian—has led to a significant improvement in model performance. The model demonstrates high precision, recall, and F1-scores for these specific cuisine categories, leading to an overall improved accuracy of 74%. This narrowing down of classes has effectively addressed the challenges associated with a large number of diverse cuisines, resulting in a more effective and interpretable model for predicting cuisine types based on ingredients.

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support

# Define parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instantiate RandomForestClassifier
rf_model = RandomForestClassifier()

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train_filtered, y_train_filtered)

# Get the best parameters and retrain the model
best_params = grid_search.best_params_
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train_filtered, y_train_filtered)

# Model Evaluation on the filtered dataset with the best model
y_pred_filtered_best = best_model.predict(X_test_filtered)
accuracy_filtered_best = accuracy_score(y_test_filtered, y_pred_filtered_best)

print(f'Best Model Accuracy on Filtered Dataset: {accuracy_filtered_best:.2f}')
print("Best Parameters:", best_params)


# Display precision, recall, and F1-score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test_filtered, y_pred_filtered_best, average=None)

class_names = best_model.classes_
for i, class_name in enumerate(class_names):
    print(f"\nClass: {class_name}")
    print(f"Precision: {precision[i]:.2f}")
    print(f"Recall: {recall[i]:.2f}")
    print(f"F1-Score: {f1_score[i]:.2f}")

Best Model Accuracy on Filtered Dataset: 0.75
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}

Class: american
Precision: 0.81
Recall: 0.51
F1-Score: 0.62

Class: british
Precision: 0.50
Recall: 0.28
F1-Score: 0.36

Class: chinese
Precision: 0.58
Recall: 0.69
F1-Score: 0.63

Class: indian
Precision: 0.81
Recall: 0.91
F1-Score: 0.86

Class: italian
Precision: 0.79
Recall: 0.75
F1-Score: 0.77


The improvement in accuracy after tuning the hyperparameters can be attributed to the optimization of the model's internal settings. By finding the best hyperparameter configuration the model is fine tuned to better capture patterns in the data and improve its ability to generalize to unseen examples.

The number of Indian cuisines is significantly larger than number of other cuisines. So the model specifically performs well on the Indian cuisine.

# **Results:**
The experiments involved two key steps to enhance model performance. First, by restricting the dataset to the top 5 cuisines, the model's accuracy showed a notable increase. This reduction in the number of classes allowed the model to focus on the most prevalent cuisines, enhancing its ability to discern patterns and relationships within the dataset. Subsequently, adjusting the class weights, specifically reducing the weight assigned to Indian cuisine, led to a further increase in accuracy. This adjustment was crucial due to the large number of Indian dishes in the dataset, ensuring a more balanced influence of different cuisines during model training. The improved accuracy post these experiments signifies that the model benefited from a more focused set of cuisines and a nuanced handling of class weights, underscoring the importance of strategic preprocessing steps in refining the model's predictive capabilities.

In [23]:
# Define class weights, giving a lower weight to the 'indian' class
class_weights = {'american': 1, 'british': 1, 'chinese': 1, 'indian': 0.2, 'italian': 1}

# Instantiate RandomForestClassifier with class weights
model_weighted = RandomForestClassifier(class_weight=class_weights, **best_params)

# Train the model on the filtered dataset with class weights
model_weighted.fit(X_train_filtered, y_train_filtered)

# Model Evaluation on the filtered dataset with the weighted model
y_pred_weighted = model_weighted.predict(X_test_filtered)
accuracy_weighted = accuracy_score(y_test_filtered, y_pred_weighted)

print(f'Weighted Model Accuracy on Filtered Dataset: {accuracy_weighted:.2f}')

# Display classification report on the filtered dataset with class weights
print(classification_report(y_test_filtered, y_pred_weighted))

Weighted Model Accuracy on Filtered Dataset: 0.78
              precision    recall  f1-score   support

    american       0.77      0.58      0.66        57
     british       0.59      0.36      0.45        36
     chinese       0.58      0.83      0.68        70
      indian       0.89      0.89      0.89       211
     italian       0.80      0.80      0.80        55

    accuracy                           0.78       429
   macro avg       0.73      0.69      0.70       429
weighted avg       0.79      0.78      0.78       429



By adjusting the class weights, specifically assigning a lower weight (0.2) to the Indian cuisine class while maintaining equal weights (1) for other top cuisines the model's accuracy experienced a notable improvement. This modification was implemented to address the imbalance in the dataset, as Indian cuisines were more prevalent than others. By assigning a lower weight to the Indian cuisine class, the model became more attuned to the distinctive features of this category, resulting in enhanced accuracy. This approach effectively prioritized the correct classification of Indian cuisines while ensuring a balanced consideration of the other selected cuisines. The refined class weights contributed to a more nuanced and accurate prediction of cuisine types, especially in scenarios with imbalanced class distributions.

# **Conclusion**

The experiments conducted on the machine learning model, specifically the reduction of cuisines to the top 5 and the adjustment of class weights for Indian dishes, yielded substantial improvements in accuracy. The decision to focus on the most prevalent cuisines allowed the model to specialize in recognizing patterns within a more concentrated set of classes, resulting in an initial boost in accuracy. Additionally, reducing the class weight for Indian dishes was instrumental in addressing the imbalance created by the large number of Indian cuisine samples. This adjustment further contributed to accuracy gains by mitigating the undue influence of Indian dishes during training. For future improvements, exploring more sophisticated techniques for handling imbalanced datasets and experimenting with hyperparameter tuning could enhance model performance. Additionally, ongoing evaluation and potential refinement of class weights based on the evolving dataset may contribute to sustained accuracy improvements. Overall, these findings underscore the importance of thoughtful preprocessing steps in tailoring the model to the characteristics of the dataset, leading to more accurate and robust predictions.